# Filtado de mensajes spam

## Descripción del problema real

La recepción de publicidad no deseada a traves mensajes de texto usando SMS (Short Message Service) es un problema que afecta a muchos usuarios de teléfonos móviles. El problema radica en que los usuarios deben pagar por los mesajes recibidos, y por este motivo resulta muy importante que las compañías prestadoras del servicio puedan filtrar mensajes indeseados antes de enviarlos a su destinatario final. Los mensajes tienen una longitud máxima de 160 caracteres, por lo que el texto resulta poco para realizar la clasificación, en comparación con textos más largos (como los emails). Adicionalmente, los errores de digitación dificultan el proceso de detección automática.

## Descripción del problema en términos de los datos

Se tiene una muestra contiene 5574 mensajes en inglés, no codificados y clasificados como legítimos (ham) o spam (http://www.dt.fee.unicamp.br/~tiago/smsspamcollection/). La información está almacenada en el archivo `datos/spam-sms.zip`.El problema en términos de los datos consiste en clasificar si un mensaje SMS es legítico o spam, a partir del análisis de las palabras que contiente, partiendo del supuesto de que ciertas palabras que son más frecuentes dependiendo del tipo de mensaje. Esto implica que en la fase de preparación de los datos se deben extraer las palabras que contiene cada mensaje para poder realizar el análsis.

## Aproximaciones posibles

En este caso, se desea comparar los resultados de un modelo de redes neuronales artificiales y otras técnicas estadísticas para realizar la clasificación.

## Requerimientos

Usted debe:

* Preprocesar los datos para representarlos usando bag-of-words.


* Construir un modelo de regresión logística como punto base para la comparación con otros modelos más complejos.


* Construir un modelo de redes neuronales artificiales. Asimismo, debe determinar el número de neuronas en la capa o capas ocultas.


* Utiizar una técnica como crossvalidation u otra similar para establecer la robustez del modelo.


* Presentar métricas de desempeño para establecer las bondades y falencias de cada clasificador.

In [18]:
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import glob
import pandas as pd
import email
import re
import sklearn

import nltk
nltk.download('punkt')
nltk.download('words')
nltk.download('stopwords')


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\jfmdd\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package words to
[nltk_data]     C:\Users\jfmdd\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\jfmdd\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [19]:
def re_html(message):
    return re.sub("<[^>]*>", "", message)

def re_websites(message):
    regex = r'^(http:\/\/www\.|https:\/\/www\.|http:\/\/|https:\/\/)?[a-z0-9]+([\-\.]{1}[a-z0-9]+)*\.[a-z]{2,5}(:[0-9]{1,5})?(\/.*)?$'
    return re.sub(regex, "", message)

def re_emails(message):
    return re.sub("[a-zA-Z0-9-_.]+@[a-zA-Z0-9-_.].+", "", message)

def re_specialCharacters(message):
    return re.sub("[^A-Za-z0-9]+", " ", message)

def re_numbers(message):
    return re.sub("[0-9]+", "", message)

def re_nonSenseWords(message):
    from nltk.corpus import words
    words = set(words.words())
    
    return " ".join(w for w in nltk.wordpunct_tokenize(message) if w in words)


In [20]:
def process_message(message, lower_case=True, stem=True, stop_words = True, gram=1):
    if lower_case:
        message = message.lower()
    words = word_tokenize(message)
    words = [w for w in words if len(w)>2]
    if gram >1:
        w=[]
        for i in range(len(words)-gram+1):
            w+=[' '.join(words[i:i+gram])]
        return w
    if stop_words:
        sw=stopwords.words('english')
        words=[word for word in words if word not in sw]
    if stem:
        stemmer=PorterStemmer()
        words=[stemmer.stem(word) for word in words]
        
    return words

In [21]:
def process_file(message):
    message = re_html(message)
    message = re_websites(message)
    message = re_emails(message)
    message = re_numbers(message)
    message = re_specialCharacters(message)
    message = re_nonSenseWords(message)
    message = process_message(message)
    return message
    

In [24]:
dataFrame = pd.DataFrame(columns =['label','message'])

data = glob.glob('./datos/spam-sms/spam-filter/*')
for name in data:
    folderName = name+'/*'
    folder = glob.glob(folderName)
    for file in folder:
        try:
            with open(file, encoding='ISO-8859-1') as f:
                message = f.read()
        except UnicodeDecodeError:
            with open(file,'rb') as f:
                message = f.read()
        dataFrame = dataFrame.append({'label':0 if 'ham' in folderName else 1, 'message': process_file(message)}, ignore_index=True)

In [8]:
allHWords = []
for p in ehm1:
    for w in p:
        allHWords.append(w)
for p in ehm2:
    for w in p:
        allHWords.append(w)
for p in hhm1:
    for w in p:
        allHWords.append(w)
for p in hhm2:
    for w in p:
        allHWords.append(w)

allSWords = []
for p in sm1:
    for w in p:
        allSWords.append(w)
for p in sm2:
    for w in p:
        allSWords.append(w)


In [13]:
from sklearn.feature_extraction.text import CountVectorizer
def bag_of_words(words):
    vectorizer = CountVectorizer()
    words = vectorizer.fit_transform(words)
    return words

In [16]:
words = bag_of_words(allHWords)
words.shape

(526683, 11527)

In [17]:
words = words.toarray()

MemoryError: 